In [2]:
# Imports
from pathlib import Path

import pandas as pd
import numpy as np
import plotly.express as px

## Pandas VS Excel

| Excel     | pandas      |
| ------------- | ------------- |
| spreadsheets | DataFrame |a
| column | Series |
| row headings | Index |
| empty cell | NaN |

[Link](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_spreadsheets.html)

In [3]:
# Same dataset as in example.xlsx

example = pd.DataFrame({
    'A': [23, 5, -12],
    'B': ['foo', 'bar', 'baz'],
    'C': ['A0', 'B0', 'C0']
})

example

,A,B,C
0,23,foo,A0
1,5,bar,B0
2,-12,baz,C0


In [6]:
example.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 101 to 103
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A       3 non-null      int64 
 1   B       3 non-null      object
 2   C       3 non-null      object
dtypes: int64(1), object(2)
memory usage: 96.0+ bytes


In [7]:
# Custom indexing

example.index = ["a", "b", "c"]
example

,A,B,C
a,23,foo,A0
b,5,bar,B0
c,-12,baz,C0


In [8]:
# Custom column names

example.columns = ['X', 'Y', 'Z']
example

,X,Y,Z
a,23,foo,A0
b,5,bar,B0
c,-12,baz,C0


In [9]:
# Selecting a column
example['X']

a    23
b     5
c   -12
Name: X, dtype: int64

In [10]:
# iloc
example.iloc[1]

X      5
Y    bar
Z     B0
Name: b, dtype: object

In [11]:
# loc
example.loc["b"]

X      5
Y    bar
Z     B0
Name: b, dtype: object

In [12]:
example.loc["a":"b"] 

,X,Y,Z
a,23,foo,A0
b,5,bar,B0


In [13]:
example.loc["c", 'Z']

'C0'

## Data about placenames in Hungary
- GeoNames dataset: https://www.geonames.org/
- (Almost) every placenames in Hungary (datasource: KSH) https://www.geonames.org/countries/HU/hungary.html: **[HU.txt](data/HU.txt)**
- Feature codes (Codes and descriptions) http://www.geonames.org/export/codes.html: **[feature_codes.tsv](data/feature_codes.tsv)**

# Data Transformation and Cleaning

In [14]:
data_path = Path("data")

In [15]:
# Load HU.txt

columns = ["geonameid",
            "name", 
            "asciiname",
            "alternatenames", 
            "latitude",
            "longitude", 
            "feature_class", 
            "feature_code", 
            "country_code", 
            "cc2",
            "admin1_code",
            "admin2_code",
            "admin3_code",
            "admin4_code", 
            "population",
            "elevation", 
            "dem", 
            "timezone",
            "modification_date"
        ]

hu_df = pd.read_csv(data_path / "HU.txt", sep="\t", header=None, names=columns, index_col=0)
hu_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24618 entries, 672629 to 12749917
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   name               24618 non-null  object 
 1   asciiname          24618 non-null  object 
 2   alternatenames     8032 non-null   object 
 3   latitude           24618 non-null  float64
 4   longitude          24618 non-null  float64
 5   feature_class      24618 non-null  object 
 6   feature_code       24618 non-null  object 
 7   country_code       24618 non-null  object 
 8   cc2                42 non-null     object 
 9   admin1_code        24584 non-null  float64
 10  admin2_code        275 non-null    float64
 11  admin3_code        0 non-null      float64
 12  admin4_code        0 non-null      float64
 13  population         24618 non-null  int64  
 14  elevation          122 non-null    float64
 15  dem                24618 non-null  int64  
 16  timezone           

In [16]:
hu_df.head()

,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,modification_date
geonameid,,,,,,,,,,,,,,,,,,
672629,Maros,Maros,"Marisus,Mures,Mures River,Muresh,Muresul,Mureş...",46.25000,20.20000,H,STM,HU,"HU,RO",0.0,NaN,NaN,NaN,0,NaN,81,Europe/Budapest,2021-02-18
677528,Gémesi Révház,Gemesi Revhaz,NaN,46.33472,18.89167,P,PPL,HU,NaN,0.0,NaN,NaN,NaN,0,NaN,102,Europe/Budapest,2014-07-08
680145,Fekete-Körös,Fekete-Koros,"Black Koros,Black Körös,Crisu Negru,Crisul Neg...",46.70000,21.26667,H,STM,HU,NaN,0.0,NaN,NaN,NaN,0,NaN,88,Europe/Budapest,2012-01-18
680147,Fehér-Körös,Feher-Koros,"Crisu Alb,Crisul Alb,Crişu Alb,Crişul Alb,Fehe...",46.93333,20.96667,H,STM,HU,NaN,0.0,NaN,NaN,NaN,0,NaN,80,Europe/Budapest,2012-01-18
680300,Kraszna-csatorna,Kraszna-csatorna,"Canalul Crasna,Canalul Crasnei,Kraszna-csatorna",48.13333,22.31667,H,CNL,HU,NaN,0.0,NaN,NaN,NaN,0,NaN,112,Europe/Budapest,2014-07-08


In [17]:
# Load feature codes

features_df = pd.read_csv(data_path / "feature_codes.tsv", sep="\t", header=None, names=["feature_code", "short_desc", "long_desc"], index_col=0)
features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 680 entries, ADM1 to VINS
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   short_desc  680 non-null    object
 1   long_desc   621 non-null    object
dtypes: object(2)
memory usage: 15.9+ KB


In [18]:
features_df.tail()

,short_desc,long_desc
feature_code,,
SCRB,scrubland,"an area of low trees, bushes, and shrubs stunt..."
TREE,tree(s),a conspicuous tree used as a landmark
TUND,tundra,"a marshy, treeless, high latitude plain, domin..."
VIN,vineyard,a planting of grapevines
VINS,vineyards,plantings of grapevines


## Null values

In [24]:
hu_df.head()

,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,cc2,population,elevation,timezone,modification_date
geonameid,,,,,,,,,,,,
672629,Maros,Maros,"Marisus,Mures,Mures River,Muresh,Muresul,Mureş...",46.25000,20.20000,H,STM,"HU,RO",0,NaN,Europe/Budapest,2021-02-18
677528,Gémesi Révház,Gemesi Revhaz,NaN,46.33472,18.89167,P,PPL,HU,0,NaN,Europe/Budapest,2014-07-08
680145,Fekete-Körös,Fekete-Koros,"Black Koros,Black Körös,Crisu Negru,Crisul Neg...",46.70000,21.26667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18
680147,Fehér-Körös,Feher-Koros,"Crisu Alb,Crisul Alb,Crişu Alb,Crişul Alb,Fehe...",46.93333,20.96667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18
680300,Kraszna-csatorna,Kraszna-csatorna,"Canalul Crasna,Canalul Crasnei,Kraszna-csatorna",48.13333,22.31667,H,CNL,HU,0,NaN,Europe/Budapest,2014-07-08


In [20]:
# Remove unnecessary columns

hu_df.drop(columns=["country_code", "admin1_code", "admin2_code", "admin3_code", "admin4_code", "dem"], inplace=True)

In [23]:
# Replace NaN with HU in cc2 column

hu_df["cc2"].fillna("HU", inplace=True)

In [25]:
# Deal with elevation

# Select rows where elevation is not null
elevations = hu_df.loc[hu_df["elevation"].notnull(), ["name", "elevation"]]

# Sort by elevation in descending order
elevations = elevations.sort_values(by="elevation", ascending=False)

elevations


,name,elevation
geonameid,,
719276,Kékes,1014.0
719996,Hidas-bérc,973.0
3052404,Galya-tető,964.0
12196238,Szilvási-kő,961.0
719745,Istállós-kő,958.0
...,...,...
11054702,Lipótváros,0.0
11054703,Hungaroring,0.0
11054704,Józsefváros,0.0


In [26]:
# View where elevation is 0
sea_level = elevations.loc[elevations["elevation"] == 0]
sea_level

,name,elevation
geonameid,,
11103601,Marmara Hotel,0.0
10942063,SAS Club,0.0
10942605,Opera Garden Hotel & Apartments,0.0
11103599,Danubius Grand Margitsziget Hotel,0.0
10942606,Opera Residence,0.0
11000645,Timon,0.0
11001704,Hotel Palota (h),0.0
11054702,Lipótváros,0.0
11054703,Hungaroring,0.0


In [27]:
# Replace 0 with NaN
hu_df.loc[hu_df["elevation"] == 0, "elevation"] = np.nan

In [30]:
hu_df.head()

,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,cc2,population,elevation,timezone,modification_date
geonameid,,,,,,,,,,,,
672629,Maros,Maros,"[Marisus, Mures, Mures River, Muresh, Muresul,...",46.25000,20.20000,H,STM,"HU,RO",0,NaN,Europe/Budapest,2021-02-18
677528,Gémesi Révház,Gemesi Revhaz,NaN,46.33472,18.89167,P,PPL,HU,0,NaN,Europe/Budapest,2014-07-08
680145,Fekete-Körös,Fekete-Koros,"[Black Koros, Black Körös, Crisu Negru, Crisul...",46.70000,21.26667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18
680147,Fehér-Körös,Feher-Koros,"[Crisu Alb, Crisul Alb, Crişu Alb, Crişul Alb,...",46.93333,20.96667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18
680300,Kraszna-csatorna,Kraszna-csatorna,"[Canalul Crasna, Canalul Crasnei, Kraszna-csat...",48.13333,22.31667,H,CNL,HU,0,NaN,Europe/Budapest,2014-07-08


## Operations on columns

In [29]:
# Convert alternatenames to list of strings

hu_df["alternatenames"] = hu_df["alternatenames"].str.split(",")

In [31]:
def check_alternatename(row):
    if row is not np.nan:
        return len(row)
    else:
        return 0

In [32]:
# Apply with function

hu_df["alternatenames_count_function"] = hu_df["alternatenames"].apply(check_alternatename)

In [33]:
# Apply with lambda function

hu_df["alternatenames_count"] = hu_df["alternatenames"].apply(lambda x: len(x) if x is not np.nan else 0)


In [34]:
hu_df[["name", "alternatenames", "alternatenames_count_function", "alternatenames_count"]].head()

,name,alternatenames,alternatenames_count_function,alternatenames_count
geonameid,,,,
672629,Maros,"[Marisus, Mures, Mures River, Muresh, Muresul,...",10,10
677528,Gémesi Révház,NaN,0,0
680145,Fekete-Körös,"[Black Koros, Black Körös, Crisu Negru, Crisul...",10,10
680147,Fehér-Körös,"[Crisu Alb, Crisul Alb, Crişu Alb, Crişul Alb,...",12,12
680300,Kraszna-csatorna,"[Canalul Crasna, Canalul Crasnei, Kraszna-csat...",3,3


## Join dataframes

- Multiple options:
  - concat()
  - merge()
  - join()

In [35]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

In [36]:
left

,A,B
K0,A0,B0
K1,A1,B1
K2,A2,B2


In [37]:
right

,C,D
K0,C0,D0
K2,C2,D2
K3,C3,D3


In [38]:
# left join
left.join(right)

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2


In [39]:
# right join
left.join(right, how="right")

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [40]:
# inner join
left.join(right, how="inner")

,A,B,C,D
K0,A0,B0,C0,D0
K2,A2,B2,C2,D2


In [41]:
# outer join
left.join(right, how="outer")

,A,B,C,D
K0,A0,B0,C0,D0
K1,A1,B1,NaN,NaN
K2,A2,B2,C2,D2
K3,NaN,NaN,C3,D3


In [42]:
hu_df.head()

,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,cc2,population,elevation,timezone,modification_date,alternatenames_count_function,alternatenames_count
geonameid,,,,,,,,,,,,,,
672629,Maros,Maros,"[Marisus, Mures, Mures River, Muresh, Muresul,...",46.25000,20.20000,H,STM,"HU,RO",0,NaN,Europe/Budapest,2021-02-18,10,10
677528,Gémesi Révház,Gemesi Revhaz,NaN,46.33472,18.89167,P,PPL,HU,0,NaN,Europe/Budapest,2014-07-08,0,0
680145,Fekete-Körös,Fekete-Koros,"[Black Koros, Black Körös, Crisu Negru, Crisul...",46.70000,21.26667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18,10,10
680147,Fehér-Körös,Feher-Koros,"[Crisu Alb, Crisul Alb, Crişu Alb, Crişul Alb,...",46.93333,20.96667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18,12,12
680300,Kraszna-csatorna,Kraszna-csatorna,"[Canalul Crasna, Canalul Crasnei, Kraszna-csat...",48.13333,22.31667,H,CNL,HU,0,NaN,Europe/Budapest,2014-07-08,3,3


In [43]:
features_df.head()

,short_desc,long_desc
feature_code,,
ADM1,first-order administrative division,a primary administrative division of a country...
ADM1H,historical first-order administrative division,a former first-order administrative division
ADM2,second-order administrative division,a subdivision of a first-order administrative ...
ADM2H,historical second-order administrative division,a former second-order administrative division
ADM3,third-order administrative division,a subdivision of a second-order administrative...


In [44]:
# Join feature codes

hu_df = hu_df.join(features_df["short_desc"], on="feature_code", how="left")

hu_df


,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,cc2,population,elevation,timezone,modification_date,alternatenames_count_function,alternatenames_count,short_desc
geonameid,,,,,,,,,,,,,,,
672629,Maros,Maros,"[Marisus, Mures, Mures River, Muresh, Muresul,...",46.25000,20.20000,H,STM,"HU,RO",0,NaN,Europe/Budapest,2021-02-18,10,10,stream
677528,Gémesi Révház,Gemesi Revhaz,NaN,46.33472,18.89167,P,PPL,HU,0,NaN,Europe/Budapest,2014-07-08,0,0,populated place
680145,Fekete-Körös,Fekete-Koros,"[Black Koros, Black Körös, Crisu Negru, Crisul...",46.70000,21.26667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18,10,10,stream
680147,Fehér-Körös,Feher-Koros,"[Crisu Alb, Crisul Alb, Crişu Alb, Crişul Alb,...",46.93333,20.96667,H,STM,HU,0,NaN,Europe/Budapest,2012-01-18,12,12,stream
680300,Kraszna-csatorna,Kraszna-csatorna,"[Canalul Crasna, Canalul Crasnei, Kraszna-csat...",48.13333,22.31667,H,CNL,HU,0,NaN,Europe/Budapest,2014-07-08,3,3,canal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12749913,Orczynegyed,Orczynegyed,[Orczy negyed],47.48541,19.08847,P,PPLX,HU,9223,NaN,Europe/Budapest,2024-03-17,1,1,section of populated place
12749914,Palotanegyed,Palotanegyed,[Palota negyed],47.49269,19.06522,P,PPLX,HU,9705,NaN,Europe/Budapest,2024-03-17,1,1,section of populated place
12749915,Századosnegyed,Szazadosnegyed,"[Szazados negyed, Százados negyed]",47.49319,19.10676,P,PPLX,HU,3774,NaN,Europe/Budapest,2024-03-17,2,2,section of populated place


In [45]:
# Number of places per feature code

feature_counts = hu_df["feature_code"].value_counts().reset_index()
feature_counts.columns = ["feature_code", "count"]
feature_counts = feature_counts.merge(features_df, on="feature_code")
feature_counts.head(20)

,feature_code,count,short_desc,long_desc
0,PPL,9819,populated place,"a city, town, village, or other agglomeration ..."
1,PPLX,5954,section of populated place,NaN
2,HLL,2217,hill,a rounded elevation of limited extent rising a...
3,AREA,1831,area,a tract of land without homogeneous character ...
4,RSTN,928,railroad station,"a facility comprising ticket office, platforms..."
5,HTL,704,hotel,a building providing lodging and/or meals for ...
6,RSTP,698,railroad stop,a place lacking station facilities where train...
7,STM,537,stream,a body of running water moving to a lower leve...
8,LK,285,lake,a large inland body of standing water
9,MT,274,mountain,an elevation standing high above the surroundi...


# Extra: Plotting geo data with Plotly

In [46]:
# Plot cities with population size

to_plot = ["Budapest", "Szeged", "Debrecen", "Miskolc", "Győr", "Pécs", "Kecskemét", "Székesfehérvár", "Szombathely", "Veszprém", "Zalaegerszeg"]

# Select Cities (feature_class = P)
p_df = hu_df.loc[hu_df["feature_class"] == "P"]

# Select cities in to_plot list
cities_df = p_df.loc[p_df["name"].isin(to_plot)]

cities_df

,name,asciiname,alternatenames,latitude,longitude,feature_class,feature_code,cc2,population,elevation,timezone,modification_date,alternatenames_count_function,alternatenames_count,short_desc
geonameid,,,,,,,,,,,,,,,
715429,Szeged,Szeged,"[Kiskundorozsma, QZD, Seged, Segedin, Seghedin...",46.25300,20.14824,P,PPLA,HU,160766,NaN,Europe/Budapest,2022-08-14,12,12,seat of a first-order administrative division
717582,Miskolc,Miskolc,"[Bukkszentlaszlo, Bükkszentlászló, MCQ, Mishko...",48.10306,20.77806,P,PPLA,HU,154521,NaN,Europe/Budapest,2022-08-14,33,33,seat of a first-order administrative division
721472,Debrecen,Debrecen,"[DEB, Debrecen, Debrecena, Debrecenas, Debrece...",47.53167,21.62444,P,PPLA,HU,202402,NaN,Europe/Budapest,2022-12-29,40,40,seat of a first-order administrative division
3042638,Zalaegerszeg,Zalaegerszeg,"[Egersee, Jegersek, Zalaederseg, Zalaegerseg, ...",46.84000,16.84389,P,PPLA,HU,61898,NaN,Europe/Budapest,2021-08-08,25,25,seat of a first-order administrative division
3042929,Veszprém,Veszprem,"[Comitatul Veszprem, Comitatul Veszprém, Gyula...",47.09327,17.91149,P,PPLA,HU,56927,NaN,Europe/Budapest,2022-08-14,38,38,seat of a first-order administrative division
3044310,Szombathely,Szombathely,"[Sabaria, Sambotel, Savaria, Sombatgej, Sombat...",47.23088,16.62155,P,PPLA,HU,78025,230.0,Europe/Budapest,2022-08-14,31,31,seat of a first-order administrative division
3044774,Székesfehérvár,Szekesfehervar,"[Alba Regalis, Alba Regia, Albareale, Sekesfeh...",47.18995,18.41034,P,PPLA,HU,101600,NaN,Europe/Budapest,2023-09-09,37,37,seat of a first-order administrative division
3046526,Pécs,Pecs,"[Cinquechiese, Fuenfkirchen, Funfkirchen, Fünf...",46.07250,18.23083,P,PPLA,HU,145347,NaN,Europe/Budapest,2023-02-09,65,65,seat of a first-order administrative division
3050434,Kecskemét,Kecskemet,"[Aegopolis, Heteny, Hetenyegyhaza, Hetény, Het...",46.90618,19.69128,P,PPLA,HU,109847,NaN,Europe/Budapest,2021-05-28,27,27,seat of a first-order administrative division


In [47]:
# Plot cities_df with plotly using population as size

fig = px.scatter_geo(cities_df, lat="latitude", lon="longitude", hover_name="name", size="population", projection="natural earth", scope="europe")

fig.show()

In [48]:
# Plot streams

stream_df = hu_df.loc[hu_df["short_desc"] == "stream"]

fig = px.scatter_geo(stream_df, lat="latitude", lon="longitude", hover_name="name", projection="natural earth", scope="europe")

fig.show()